In [1]:
!pip install textract transformers

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
!pip install sentencepiece

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
import torch

if torch.cuda.is_available():
    print("GPU is available")
    device = torch.device("cuda")
else:
    print("GPU not available, using CPU")
    device = torch.device("cpu")

GPU not available, using CPU


In [4]:
def merge_ner_results(ner_results):
    merged_results = []
    current_entity = None

    for result in ner_results:
        word = result['word'].replace('▁', ' ')
        if current_entity and result['entity'].startswith('I-') and current_entity['entity_group'] == result['entity'][2:]:
            current_entity['word'] += word
            current_entity['end'] = result['end']
            current_entity['score'] = (current_entity['score'] + result['score']) / 2
        else:
            if current_entity:
                merged_results.append(current_entity)
            current_entity = {
                'entity_group': result['entity'][2:],
                'score': result['score'],
                'word': word,
                'start': result['start'],
                'end': result['end']
            }

    if current_entity:
        merged_results.append(current_entity)

    return merged_results


In [5]:
def chunk_text_to_tokens(text, max_tokens):
    # Tokenize the text and get token ids
    tokens = tokenizer_summary.encode(text, add_special_tokens=False)

    # Chunk token ids
    token_chunks = [tokens[i:i + max_tokens] for i in range(0, len(tokens), max_tokens)]

    # Convert token chunks back to text
    chunk_texts = [tokenizer_summary.decode(chunk, skip_special_tokens=True, clean_up_tokenization_spaces=True) for chunk in token_chunks]
    return chunk_texts

In [6]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModel, AutoModelForSeq2SeqLM

#NER Model
model = AutoModelForTokenClassification.from_pretrained("cahya/xlm-roberta-large-indonesian-NER")
tokenizer = AutoTokenizer.from_pretrained("cahya/xlm-roberta-large-indonesian-NER")
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

# Summarize Model
model_summary_name = "cahya/bert2gpt-indonesian-summarization"
model_summary = AutoModelForSeq2SeqLM.from_pretrained(model_summary_name)
tokenizer_summary = AutoTokenizer.from_pretrained(model_summary_name)

# Move model to GPU
model_summary.to(device)
summarizer = pipeline("summarization", model=model_summary, tokenizer=tokenizer_summary) # device=0 for the first GPU


#Given Text
#text = "Nama Saya Arief Kurniawan, saat ini tanggal 29 November 2023, di Bogor, Sedang bekerja dengan cara menghadiri rapat pada Pukul 08.00 WIB dengan membawa uang sebesar Rp 10.000.000 dan 5 buah tas."

text = "A.N. Angeligue Hayyuza, Se Dan Novel Mohamad Mahri 1. Ringkasan Kasus Angeligue Hayyuza (Ah) Dilaporkan Oleh Pedagang Valuta Asing, Pt Dinamis Citra Swakarsa Terkait Transaksi Penjualan Bank Notes Dalam Jumlah Besar. Berdasarkan Informasi Pt Dinamis Citra Swakarsa, Ah Berprofesi Sebagai Pegawai Swasta Namun Tidak Diketahui Nama Dan Domisili Perusahaan Tempat Ybs Bekerja. Sesuai Data Nasabah Dari Bank Bca Terinformasi Bahwa Ah Berprofesi Sebagai Pedagang (Istri Pemilik Toko Himalaya-Toko Bangunan Berdasarkan Data Bank Mandiri, Pekerjaan Ah Adalah Ibu Rumah Tanggaterinformasi Juga Media Online, Sejak Tahun 2012, Ah Adalah Pegawai Pada Perusahaan Money Remittance (Nusantara Money Remittance) Yang Berdomisili Operasi Di Sydney Australiaselama Periode Mei 2012 S/D Juli 2013 (15 Bulan Teridentifikasi Ah Melakukan 40 Kali Transaksi Penjualan Bank Notes Dengan Total Nominal Aud 571,447 (Eg Rp5.308.610.800)Atau Rata-Rata Ah Melakukan 3 Kali Transaksi Per Bulanah Memiliki 3 (Tiga) Rekening Di Bank Mandiri Dan Bank Bca. Berdasarkan Mutasi Rekening Bca Nomor 2810297291 A.N Ybs, Selama Periode Januari 2012 S/D April 2013, Ah Menerima Aliran Dana Sebanyak 443 Kali Sejumlah Total Rp29.663.595.676, Termasuk Setoran Tunai Senilai Total Rp718.710.000. Adapun Aliran Dana Keluar Berupa 1.872 Kali Transaksi Pengiriman Dana Total Rp17.434.835.607. (Rata-Rata Sebesar Rp9.313.480 Per Transaksikepada Banyak Pihak Dengan Underlying Nusantara Diduga Nusantara” Adalah Nusantara Money Transfer, Perusahaan Tempat Ybs Bekerjaberdasarkan Mutasi Rekening Bank Mandiri Nomor 9000003185833, Pada Periode Januari 2012 S/D Maret 2013, Ah Aktif Menerima 102 Kali Transaksi Aliran Dana Berupa Pemindahbukuan Dari Seseorang Bernama Novel Mohamad Mahri (Nmm) Dengan Total Nominal Rp3.213.127.075. Adapun Aliran Dana Keluar Berupa 855 Kali Transaksi Pengiriman Dana Total Rp6.922.841.034,Pnii Ii Gi N Ii Magi I A Ama Ai Gnii Inng Pennan/ E Pn Jii Mgii Jcugiiixiiyg. Peni Na Ii Nga Si Vv Ner/ Pe Aag Iiiciiycuulixgiitransaksi Pada Kanggaru Bisa Menggunakan 4 (Empat) Rekening Bank Atas Nama Nmm Masing-Masing Di Bank Mandiri, Bank Bca, Bank Bni, Bank Briselama Periode 01 Januari 2012 S/D 30 Maret 2013, Melalui Rekening Bank Mandiri Nomor 1400004357068, Nmm Menerima Aliran Dana Masuk Yang Berasal Dari Setoran Tunaioverbooking, Rtgs Dll Dari Beberapa Pihak Sebanyak 2.423 Kali Transaksi Dengan Total Nominal Rp152.345.554.931. Dari Jumlah Tersebut, Aliran Dana Yang Berasal Dari Ybs Sendiri (Nmm) Mencapai 202 Kali Transaksi Berupa Setoran Tunai Dan Rtgs, Total Nominal Rp50.910.900.000. Teridentifikasi Untuk Aliran Dana Keluar, Nmm Mengirimkan Dana Ke Banyak Pihak (13.956 Kali Transaksi, Total Nominal Rp151.375.094.282 Berdasarkan Pola Mutasi (Debit Vs Kredit) Transaksi Yang Bersifat From-Many-To-Many” Diindikasikan Terkait Dengan Kegiatan Remitansi Pada Kupu Kanggaruinformasi Dari Website Www.Kanggaru.Net, Kanggaru Merupakan Perusahaan Terdaftar Di Remitansi Kupu Kanggaru Dan Jaringannya Di Indonesia. Seharusnya, Kupu Kanggaru Menggunakan Rekening Perusahaan Untuk Transaksi Remitansi Tersebutatas Dasar Pertimbangan Tersebut Maka Hasil Analisis Atas Nama Angeligue Hayyuza Dan Novel Mohamad Mahri Diteruskan Kepada Direktorat Jenderal Pajak Untuk Ditindaklanjuti Sesuai Kewenangan Yang Berlaku Serta Menjadi Informasi Kepada Bank Indonesia Terkait Fungsi Pengawasan Dan Pembinaan Terhadap Industri Kupu 2. Profil Terlapor Dan Terkait Serta Pola Transaksi A. Profil Terlapor Nama : | Angeligue Hayyuza (Ahpep : | Tidak Jenis Kelamin : | Perempuan Tempat, Tgl Lahir : | 29 November 1984 Alamat : | Jl Saturnus Utara Ix No 1 Rt 02 Rw 11 Munjahlega Rancasari Bandung Jawa Barat 40286 Pekerjaan : | Ibu Rumah Tangga (Data Nasabah Bank Mandiripegawai Swasta (Karyawan Nusantara Money Transfer Perusahaan Money Remittance) — Informasi Dari Media Online Pedagang (Istri Pemilik Toko Himalaya-Toko Bangunan) Data Nasabah Bca Identitas Diri : | Ktp No. 1050216911843001 Alamat Tempat Kerja Nusantara Money Transfer, 52/3-7Fetherstone St Bca Nomor 0080957700 A.N Angeligue Hayyuza Bank Mandiri 9000003185833 A.N Angeligue Hayyuza Nama : | Novel B Mohammad Mahri (Nmmpep : | Tidak Jenis Kelamin : | Laki-Laki Kepemilikan Dana : | Pemilik Dana Kewarganegaraan : | Indonesia Tempat, Tgl Lahir : | Surabaya, O6 November 1962 Alamat : | Ampel Wirai 2/4 Rt 002 Rw 003, Ampel Lonceng 27Ampel, Semampir Surabaya - Jawa Timur 60151 No Telp 081553793218 Email Nmahri@Optusnet.Com.Au Alamat Tempat Kerja Kanggaru Money Transfer, Surabaya Jawa Timur Norek Yang Digunakan Bank Mandiri 1400004357068 A.N Nmm Bca 0360198774 A.N Nmm Bni 0120862977 A.N Nmm Bri 105 101000 081 503 A.N Nmm Nama Nusantara Money Transfer (Nmtbidan Nin Fe Docwne ( Was Created Wiih Spile.Doc For Py Registrasi Fair Trading Registration : Bn 9855 Pendiri Muhammad Basuki Alamat Surat 52/3-7Fetherstone St, Bankstone, Nsw Telp 0401-637278 Email Info@Mynusantara.Com Remittance Rekanan Di Pt Warindo Indonesia Nama Pt Warindo (Wabidang Usaha Pengiriman Uang (Mmoney Remittancealamat Graha Mas Fatmawati A-30, Ji Rs. Fatmawati No.71 Jakarta 12150 Honemail : | Pt.Warindo@Yahoo.Com Website : | Http:Www.Warindo.Co.Id Nama : | Kanggaru Money Transfer Bidang Usaha : | Pengiriman Uang (Mmoney Remittanceregistrasi : | Abn(770792 937 159) Dan Can (117 567 110Pendiri Ie Alamat Di Australia : | 11Marsham St. Noble Park Victoria 3174 Australia Telp : | 61402181266 /4 Surabaya 60151 Telp : | 031-3550844, 031-3545708, 62 815537932183 Email : | Thahri@Kanggaru.Net B. Pola Transaksi 1. Berdasarkan Data Pt Dinamis Citra Swakarsa, Selama Periode Mei 2012 S/D Juli 2013 (15 Bulan Ah Melakukan 40 Kali Transaksi Penjualan Bank Notes Dengan Rincian Sbb: Transaksi Jumlah Rupiah N Tgl T Ksi Rekeni I K ) Gi Transaksi Valas (Rp) Ekening Yang Digunakan Bca 2810297291 1 1 31/07/2013 Aud 19,195 178.513.500,Mandiri 9000003185833 2 24/07/2013 Aud 13,145 123.225.000 | Bca 2810297291 3 18/07/2013 Aud 18,360 169.802.250 | Bca 2810297291 4 16/07/2013 Aud 14,870 136.051.350 | Bca 2810297291 5 /109/07/2013 Aud 32,110 290.577.400 | Bca 2810297291 E— Nij Na Nj G N Da Gi P Maa M — Ie M A M My 7 I 8 Om 7 A / Om T S Gu O 7 | 19/06/2013 Aud 11,125 103.462.500 | Mandiri 9000003185833 Bca 2810297291 8 | 31/05/2013 Aud 12,465 117.756.400,Mandiri 9000003185833 9 / 06/05/2013 Aud 24,045 239.237.800 | Bca 2810297291 10 | 01/05/2013 Aud 15,515 155.130.000 | Bca 2810297291 11 | 08/03/2013 Aud 11,791 116.436.125 | Bca 2810297291 12 | 28/02/2013 Aud 10,112 99.856.000 | Bca 2810297291 13 | 15/01/2013 Aud 14,023 144.436.900 | Bca 2810297291 14 | 14/01/2013 Aud 15,705 162.154.125 | Bca 2810297291 Bca 2810297291 15 | 09/01/2013 Aud 9,543 97.577.175,Mandiri 9000003185833 Bca 2810297291 16 | 08/01/2013 Aud 14,331 146.176.200,Mandiri 9000003185833 : The Document Was Created With &Elre'Doc For Python21/12/2012 2810297291 17 Aud 10,471 105.757.100, Mandiri 9000003185833 Bca 2810297291 18 | 20/12/2012 Aud 4,933 49.823.300,Mandiri 9000003185833 Bca 2810297291 19 | 19/12/2012 Aud 6,483 65.478.300,Mandiri 9000003185833 Bca 2810297291 20 | 17/12/2012 Aud 5,984 61.347.400,Mandiri 9000003185833 Bca 2810297291 21 | 14/12/2012 Aud 11,975 120.947.500 Mandiri 9000003185833 Bca 2810297291 22 | 13/12/2012 Aud 28,310 7171.689.200,Mandiri 9000003185833 Bca 2810297291 23 | 12/12/2012 Aud 19,654 198.505.400,Mandiri 9000003185833 Bca 2810297291 24 | 11/12/2012 Aud 19,602 197.000.100 Mandiri 9000003185833 25 | 06/12/2012 Aud 8,834 88.340.000 Bca 2810297291 Mandiri 9000003185833 7 I 8 Om 7 A / Om T S Gu O 26 | 05/12/2012 Aud 13,765 137.650.000,Mandiri 9000003185833 27 | 04/12/2012 Aud 5,497 54.695.150,00 | Mandiri 9000003185833 28 | 03/12/2012 Aud 5,098 50.725.100 | Bca 2810297291 Bca 2810297291 29 | 21/11/2012 Aud 10,592 105.390.400,Mandiri 9000003185833 Bca 2810297291 30 | 20/11/2012 Aud 15,565 154.871.750,Mandiri 9000003185833 Bca 2810297291 31 | 19/11/2012 Aud 15,879 157.202.100 | N Ar 9000003185833 Bca 2810297291 32 | 12/11/2012 Aud 12,012 119.519.400 | N Ar 9000003185833 33 | 13/11/2012 Aud 15,970 158.901.500 | Bca 2810297291 34 | 07/08/2012 Aud 21,571 213.552.900 | Bca 2810297291 . . . %A 2810297291 Bca 2810297291 36 | 20/06/2012 Aud 21,612 205.314.000 | Na 9000003185833 37 | 19/06/2012 Aud 12.216 115.441.200 | Mandiri 9000003185833 38 | 18/06/2012 Aud 12,852 121.451.400 | Bca 2810297291 39 | 28/05/2012 Aud 12,834 117.431.100 | Bca 2810297291 40 | 23/05/2012 Aud 9,945 91.742.625 | Bca 2810297291Total 571,447 5.308.610.800,2. Selama Periode Mei 2012 S/D Juli 2013 (15 Bulan) Dengan Total Nominal Aud 571,447 (Eg Rp5.308.610.800 Rata-Rata Ah Melakukan 3 Kali Transaksi Per Bulan3. Ah Diketahui Memiliki Tiga Rekening Di Bca, Namun Hanya Dua Rekening Yang Masih Aktif. Ah Memiliki Fasilitas Kartu Kredit Yang Dibuka Sejak Tanggal 3 Oktober 2012 Dengan Limit Kredit Sebesar Rp5.000.000, Namun Ybs Tidak Memiliki Fasilitas Pinjaman, Sdb Dan Ybs Juga Bukan Merupakan Nasabah Kustodian Di Bca. Rincian Kepemilikan Rekening Yang Dimiliki Ah Sbb: 2800539881 Bca Kco Kiaracondong | 16/06/2005 19/05/2010 2810297291 Bca Kcp Rajawali 12/12/2011 39.670.698,47 0080957700 Bca Kcp Rajawali 07/05/2013 1.472.091,72 Berdasarkan Mutasi Rekening Nomor 2810297291, Selama Periode Januari 2012 S/D April 2013, Ah Menerima Dana Masuk Dari Pt Dinamis Citra Swakarsa (Perusahaan Money Changer) Sebanyak 65 Kali Transaksi, Total Rp6.535.337.505. Ah Juga Menerima Dana Dari Novel M Mahri Sebanyak 30 Kali Transaksi Total Sebesar Rp1.036.571.715. Diduga Pengiriman Ini Terkait Dengan Bisnis Yang Dimiliki Nmm Yaitu Bisnis Pengiriman Uang (Mmoney Remittance)4. Sementara Aliran Dana Keluar Dari Rekening Ah Berupa Transaksi Pengiriman Dana Ke Banyak Pihak Dengan Underlying Nusantara Diduga Nusantara” Merupakan Nusantara Money Transfer, Perusahaan Tempat Ybs Bekerja. Selama Periode Januari 2012 S/D April 2013 (16 Bulan Ah Mentransfer Dana Sebangk 1.872 Kali Dengan Total Ire.Doc For Xthondana Sebesar Rp17.434.835.607. (Rata-Rata Sebesar Rp9.313.480 Per Transaksi)Aliran Dana Keluar Dari Rekening Ah Lainnya Berupa Transfer, Pembelian Barang, Tarikan Tunai Sebanyak 149 Kali Dengan Total Rp2.776.348.200Transaksi Aliran Dana Keluar Dari Rekening Bca Nomor 2810297291 Ah Periode Januari 2012 S/D April 2013 Dengan Dengan Nominal Lebih Dari Rp50.000.000 Sbb: D Tgl / Jenis Transaksi Jumlah Nama Pengirim/Penerima Keterangan Berita K Pemindahan Novel Mohamad 1-Feb-12 | K | Langsung Dari 268,403,600.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Langsung 1-Feb-12 | D | Tabungan Ke 94,500,000.00 | Pwe Tjhai Trsf Ke 0290336457 | Pwe Tjhai Tahapan Pemindahan Wierman Liduan Wierman 1-Feb-12 | D | Langsung 94,500,000.00 | (Pemilik Toko Naga Trsf Ke 6430057799 | Liduan Tabungan Ke Giro Komodo - Alat Rumah Tanggapemindahan Novel Mohamad 2-Feb-12 | K | Langsung Dari 323,273,900.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Novel Mohamad 3-Feb-12 | K | Langsung Dari 326,790,450.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Langsung 3-Feb-12 | D | Tabungan Ke 94,500,000.00 | Gui Lie Tju Trsf Ke 0931070508 | Guilie Tju Tahapan Pemindahan Langsung 3-Feb-12 | D | Tabungan Ke 94,500,000.00 | Pwe Tjhai Trsf Ke 0290336457 | Pwe Tjhai Tahapan Pemindahan Novel Mohamad 6-Feb-12 | K | Langsung Dari 180,774,975.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Wierman Liduan Wierman 6-Feb-12 | D | Langsung 142,875,000.00 | (Pemilik Toko Naga Trsf Ke 6430057799 | Liduan Tabungan Ke Giro Komodo - Alat Rumah Tanggatarikan Tunai 6-Feb-12 | D | Dgnbuku Kecil 53,676,000.00 (Thp Goldpemindahan Novel Mohamad 7-Feb-12 | K | Langsung Dari 122,409,000.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 . . Hapan I 8-Feb-12 | K | Langsung Dari 133,868,950.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Novel Mohamad 10-Feb-12 | K | Langsung Dari 163,382,350.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Wierman Liduan Wierman 10-Feb-12 | D | Langsung 95,500,000.00 | (Pemilik Toko Naga Trsf Ke 6430057799 | Liduan Tabungan Ke Giro Komodo - Alat Rumah Tanggadindin B Zakaria (Karyawan Ptsrikania 10-Feb-12 | D | Iransfer Via Ib 57,300,000.00 | Cimandiri - Nusantara Ke Tahapan Engineering Automotive Partspemindahan Novel Mohamad 13-Feb-12 | K | Langsung Dari 58,316,150.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dsetoran 14-Feb-12 | K | Pemindahandgn | 141,288,700.00 Berita Pemindahan Novel Mohamad 16-Feb-12 | K | Langsung Dari 106,258,400.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dtransfer Via Ib 16-Feb-12 | D | Ke Tahapan 50,302,500.00 | Budi Horno Teja Ke Nusantara Pemindahan Novel Mohamad 20-Feb-12 | K | Langsung Dari 152,453,150.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&D22-Feb-12 | K | Langsung Dari 99,199,000.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Novel Mohamad 24-Feb-12 | K | Langsung Dari 88,406,400.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Novel Mohamad 27-Feb-12 | K | Langsung Dari 73,152,000.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Novel Mohamad 28-Feb-12 | K | Langsung Dari 98,402,150.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Novel Mohamad 29-Feb-12 | K | Langsung Dari 80,212,000.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Novel Mohamad 2-Mar-12 | K | Langsung Dari 211,687,100.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Langsung Ricy 2-Ma-12 | D | Tabungan Ke 72,750,000.00 | Ricy Candra Trsf Ke 3360020648 | Candra Tahapan Pemindahan Novel Mohamad 5-Mar-12 | K | Langsung Dari 153,987,600.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 . . Hapan 5-Ma-12 | D %Zggflgxg Ke 130,450,000.00 Trsf Ke 4740620121 Mtjjsaunrt Tahapan Karjanti Mansur (Ibu Rumah Tanggakoreksi Tarikan 5-Mar-12 | K | Pemindahan Tnp 67,725,000.00 Sal Nominal 627 25000 Buku Transfer Via Ib 5-Ma-12 | D | Ee Tahapan 57,900,000.00 | Gui Lie Tju Nusantara Aus Pemindahan Novel Mohamad 7-Mar-12 | K | Langsung Dari 60,703,900.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Novel Mohamad 9-Mar-12 | K | Langsung Dari 75,861,200.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Novel Mohamad 13-Mar-12 | K | Langsung Dari 56,756,450.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Novel Mohamad 14-Mar-12 | K | Langsung Dari 89,925,100.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dtransfer Via Ib Karjanti Mansur (Ibu 14-Mar-12 D 61,120,000.00 Nusantara Aus Ke Tahapan Rumah Tanggapemindahan Novel Mohamad 16-Mar-12 | K | Langsung Dari 53,277,750.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Novel Mohamad 19-Mar-12 | K | Langsung Dari 123,086,000.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&D10"
jumlah_kata = len(text.split())
print(f"\nJumlah kata dalam teks asli: {jumlah_kata}")

# Chunk the text
max_tokens = 500  # Adjust as needed to be below the model's token limit
text_chunks = chunk_text_to_tokens(text, max_tokens)

# Summarize each chunk and combine
summaries = [summarizer(chunk)[0]['summary_text'] for chunk in text_chunks]
combined_summary = ' '.join(summaries)

# NER text
ner_results = ner_pipeline(text)
ner_results

#Merge NER
merged_results = merge_ner_results(ner_results)
merged_results


Some weights of the model checkpoint at cahya/xlm-roberta-large-indonesian-NER were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The encoder model config class: <class 'transformers.models.bert.configuration_bert.BertConfig'> is different from the decoder model config class: <class 'transformers.models.gpt2.configuration_gpt2.GPT2Config'>. It is not recommended to use the `AutoTokenizer.from_pretrained()` method in this 


Jumlah kata dalam teks asli: 2222


[{'entity_group': 'PER',
  'score': 0.9870443451218307,
  'word': ' A.N. Angeligue Hayyuza',
  'start': 0,
  'end': 22},
 {'entity_group': 'PER',
  'score': 0.6766662,
  'word': ' Se',
  'start': 23,
  'end': 26},
 {'entity_group': 'PER',
  'score': 0.9658958464860916,
  'word': ' Novel Mohamad Mahri',
  'start': 30,
  'end': 50},
 {'entity_group': 'PER',
  'score': 0.9961642473936081,
  'word': ' Angeligue Hayyuza',
  'start': 69,
  'end': 87},
 {'entity_group': 'PER',
  'score': 0.8584772,
  'word': 'Ah',
  'start': 89,
  'end': 91},
 {'entity_group': 'ORG',
  'score': 0.9984800284728408,
  'word': ' Pt Dinamis Citra Swakarsa',
  'start': 131,
  'end': 157},
 {'entity_group': 'PRD',
  'score': 0.8765986561775208,
  'word': ' Bank Notes',
  'start': 185,
  'end': 196},
 {'entity_group': 'ORG',
  'score': 0.9984336476773024,
  'word': ' Pt Dinamis Citra Swakarsa',
  'start': 238,
  'end': 264},
 {'entity_group': 'PER',
  'score': 0.9970831,
  'word': ' Ah',
  'start': 265,
  'end': 268

In [9]:
print("Summary:\n", combined_summary)
jumlah_kata_summary = len(combined_summary.split())
print(f"\nJumlah kata dalam teks summary: {jumlah_kata_summary}")


Summary:
 kasus angelimue hayyuza dilaporkan oleh pedagang valuta asing, pt dinamis citra swakarsa. ah berprofesi sebagai pegawai swasta namun tidak diketahui nama dan domisili perusahaan tempat ybs di bank mandiri, bank bca, bank bni, bank nasabah menerima aliran dana masuk yang berasal dari setoran tunaieringbooking, rtgs leks dari beberapa pihak sebanyak 135 kali transaksi. surabaya, o6 november 1962 alamat : surabaya, jawa timur, norek yang digunakan bank mandiri ump transfer, surabaya jawa timur 60151 no telp. com. rabu ( 11 / 07 ), bca 28 / 08 / 2005 ( 1 / 11 ), kamis ( 21 / 10 ), dan 15 / 09 / 10 / berikutnya. di antaranya, rabu ( 12 / 12 ), bca 2810kana7247241 20 17 /, 984 61. 347. 400, mandiri tiket urut. selama mei 2012 s / d, rabu ( 19 / 06 / 06 ), merupakan dua rekening di bca, namun hanya dua rekening yang masih aktif. sementara aliran dana yang dibuka sejak rabu 2012 aliran dana keluar dari rekening ah lainnya berupa transfer, pembelian barang, tarikan tunai sebanyak 149 

In [10]:
# Extracting 5W1H information
who = []
when = []
where = []
what = []
how_much = []
for entity in merged_results:
    if (entity['entity_group'] == 'PER') | (entity['entity_group'] == 'ORG') | (entity['entity_group'] == 'NOR'):
        who.append(entity['word'])
    elif (entity['entity_group'] == 'LOC') | (entity['entity_group'] == 'GPE') | (entity['entity_group'] == 'FAC') | (entity['entity_group'] == 'REG'):
        where.append(entity['word'])
    elif (entity['entity_group'] == 'TIM') | (entity['entity_group'] == 'DAT'):
        when.append(entity['word'])
    elif (entity['entity_group'] == 'EVT') | (entity['entity_group'] == 'PRC'):
        what.append(entity['word'])
    elif (entity['entity_group'] == 'MON') | (entity['entity_group'] == 'QTY'):
        how_much.append(entity['word'])
print("Who:", ", ".join(who))
print("When", ", ".join(when))
print("Where:", ", ".join(where))
print("What:", ", ".join(what))
print("How Much:", ", ".join(how_much))

Who:  A.N. Angeligue Hayyuza,  Se,  Novel Mohamad Mahri,  Angeligue Hayyuza, Ah,  Pt Dinamis Citra Swakarsa,  Pt Dinamis Citra Swakarsa,  Ah,  Bank Bca,  Ah, To,  Bank Mandiri,  Ah,  Media,  Ah,  Money Remittance, Nusantara Money Remittance,  Ah,  Ah,  Bank Mandiri,  Bank Bca,  Ah,  Nusantara,  Nusantara Money Transfer,  Ah,  Novel Mohamad Mahri, Nmm, P, ni
When  2012,,  Mei 2012 Juli 2013,  Januari 2012,  April 2013,,  Januari 2012,  Maret 2013,
Where:  Sydney Australia,  Nusantara
What: 
How Much:  (15 Bulan,  A, ud 571,447,  Rp5.308.610.800,  Rp29.663.595.676,  Rp718.710.000,  Rp17.434.835.607,  Rp9.313.480,  Rp3.213.127.075,  Rp6.922.841.034


In [12]:
# tes kedua
from datetime import datetime

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

text ="Transaksi Keuangan Mencurigakan An. Abu Huraeroh 1. Kasus Posisi Terinformasi Abu Huraeroh/Ah (49 Tahun) Adalah Pegawai Negeri Sipil Pada Pemerintah Provinsi Dki Dengan Jabatan Kasubag Keuangan. Adapun Para Pihak Terkait Transaksi Keuangan Ah Adalah Nina Chairunissa/Cn (Istri Ah) Dan Banyak Pihak, Baik Perusahaan Antara Lain Pt Amalgam Indocorporapt Trinusa Aneka Tambang Dan Perseorangan Antara Lain Aisyawati Suseno, Evi Muliyani, Hambalilie Sin Hwa, Ismeth Abdullah, Lufti Khoir, Novel Ali, Safrin Rachman, Siti Chair Rachman, Tambayongtourik Mahriberdasarkan Penelusuran Transaksi Keuangan Ah Dan Pihak Terkait Untuk Periode Januari 2004 Sampai Dengan Desember 2013, Diketahui: E Ahteridentifikasi Memiliki 16 (Enam Belas) Rekening Di Bca, Bni, Bank Mandiri, Bank Danamon Syariah, Bank Muamalat Indonesia, Btn Dan Bank Panin Dengan Total Dana Masuk Pada Ke-16 Rekening Dimaksud Setidaknya Mencapai Rp78,72 Miliar Termasuk Diantaranya Sejumlah Setoran Tunai Dan Transfer Yang Berasal Dari Ah Sendiri. Periode Januari 2004 Sampai Dengan Desember 2013, Teridentifikasi Ah Melakukan/Menerima Setoran Tunai Setidaknya Sebesar Rp30,85 Miliare Adapun Dana Keluar Pada Rekening Ah, Pada Umumnya Berupa Tarik Tunai Dan Transfer Ke Rekening Ah Lainnya Dan Ke Banyak Pihak. Sebagian Besar Teridentifikasi Digunakan Untuk Transaksi Pembelian Tanah, Rumah, Kendaraan Bermotor (Baik Dilakukan Langsung Maupun Melalui Balai Lelang)Berkenaan Dengan Aliran Penerimaan Dana Direkening Ah Yang Setidaknya Mencapai Rp78,72 Miliartermasuk Setoran Tunai Senilai Total Rp30,85 Miliar, Dan Aktivitas Pembelian Tanah, Rumah, Dan Kendaraan Bermotor Serta Transaksi Dengan Underlying Pinjam Meminjam/Hutang Piutang Dalam Nominal Yang Signifikan Merupakan Transaksi Keuangan Yang Tidak Wajar Karena Menyimpang Dari Profil Ah Sebagai Pns Pada Pemerintah Provinsi Dki. Sampai Dengan Saat Ini, Tidak Terinformasi Mengenai Adanya Pekerjaan Atau Kegiatan Usaha Lainnya Yang Terkait Atau Menjadi Underlying Atas Sejumlah Transaksi Keuangan Tersebut Diatas Sehingga Asal Usul Kekayaan Ah Menjadi Tidak Jelasatas Hal Tersebut, Diperlukan Penyelidikan Dan Atau Penyidikan Lebih Lanjut Untuk Mengetahui Underlying/Tujuan Transaksi Ah Dengan Para Pihak Tersebut Untuk Membuktikan Ada Atau Tidaknya Transaksi Ah Terkait Dengan Suatu Tindak Pidana Korupsi/Gratifikasi Yang Berkaitan Dengan Tugas Dan Tanggung Jawab Ah Sebagai Pegawai Pemprov Dki2. Profil Para Pihak Nama : Abu Huraeroh Jenis Kelamin : Laki-Laki Nomor Telepon 021-83791318, 08161948564 0925010107650463, 09.5301.010765.0463 (Ktp650712051390 (Simpns Pada Kantor Walikota Jakarta Selatan (Btn, Bminomor Identitas Pns - Pemda Dki Jakarta (Bnipekerjaan Pns - Kabag Keuangan Pemda Dki (Bca, Paninpns Kementrian Agama (Bank Mandirirekening An. Ah Rekening Bank Nama Keterangan 1102030916 Panin Abu Huraeroh | Saldo Per 24/07/2014 Rp731.985,71 150534508 Bni Abu Huraeroh | Tutup Per 31 Januari 2014 146465110 Bni Abu Huraeroh | Tutup Per 8 Juli 2009 0051033751 Bca Abu Huraeroh | Tutup Per 13/12/2002 2721223967 Bca Abu Huraeroh | Saldo Per 13/09/2013 Rp1.850.995.698,83 2721339701 Bca Abu Huraeroh | Saldo Per 13/09/2013 Rp3.972.305,19 0046301500011046 Btn Abu Huraeroh | Saldo Per 31/12/13 Rp4.838.386.910 0162570678 Muamalat Abu Huraeroh | Saldo Per 30/01/14 Rp 103-00-05550179 Mandiri Abu Huraeroh | Rekening Haji 124-00-99623408 Mandiri Abu Huraeroh | Saldo Per 31/12/13 Rp376.546.778,73 006-00-04335810 Mandiri Abu Huraeroh | Tutup Per 31/05/2004 123-00-04232023 Mandiri Abu Huraeroh | Tutup Per 02/01/2008 124-00-05282158 Mandiri Abu Huraeroh | Tutup Per 22/11/2010 124-00-04238664 Mandiri Abu Huraeroh | Saldo Per 13/02/2014 Rp565.001.349 Nama : Nina Chairunnisa Jenis Kelamin Ttl : Jakarta, 10/10/1970 Jln. Kampung Melayu Kecil, Rt.O08 Rw.09, Kelurahan Bukit Perempuan Alamat Duri, Kecamatan Tebet, Jakarta Timur Jl. Batu Tulis Vi No. 39, Rt.O11 Rw.003, Kebon Kelapaalamat 2 Jakarta Nomor Identitas 3174015010700005 Pekerjaan Rekening An. Nc: Sekretaris Pt Ism Bogasari Flour Mills (Industri Makananzi Zlujtuzil Dla Inina Yilair Unntila Lanapari 4581616583 Bca Nina Chairunnisa | Tapres (Rdi Pt Indopremier Securities5409120010609602 | Bca Nina Chairunnisa | Kartu Kredit 1888801367115504 | Bca Nina Chairunnisa | Kartu Kredit 4546335300345082 | Permata Nina Chairunnisa | Kartu Kredit 5189439018687600 | Permata Nina Chairunnisa | Kartu Kredit 3. Pola Transaksi A. Terkait Rekening Atas Nama Ah, Dapat Dijelaskan Sebagai Berikut: 1) Rekening 1102030916 Atas Nama Ah Di Bank Panin Kcp Tebet Dibuka Pada Tanggal 20 Desember 2011 Dengan Setoran Awal Berupa Setor Tunai Sebesar Rp1 Juta. Pada Tanggal 22 Desember 2011 Terdapat Rtgs Masuk Sebesar Rp1,7 M Dari Rekening Ybs Di Bca Cabang Gudang Peluru Nomor 2721339701 Dan Setor Tunai Oleh Ybs Sebesar Rp115 Juta. Pada Hari Yang Sama Dana Masuk Tersebut Di Transfer Ke Rekening Atas Nama Arum Mawarti Masingmasing Sebesar Rp350 Juta Dan Rp1,46 M, Dengan Keterangan Transaksi Untuk Pembayaran Shm No.809/Gandul. Selanjutnya, Periode Desember 2011 Sampai Dengan Juli 2014 Rekening Tidak Aktif, Hanya Terdapat Transaksi Administrasi Dan Bunga. Saldo Rekening Per Tanggal 24 Juli 2014 Adalah Sebesar Rp731.985,71 Tidak Terlalu Aktif. Namun Demikian Terdapat Beberapa Transaksi Yang Dilakukan Dengan Nominal Relatif Signifikan Diatas Rp100 Juta, Yaitu: Tanggal D/K Nominal Keterangan 06/05/2008 K 500.000.000,00 | Transfer Dari Tutup Deposito Ab270329 Dari Rekening 95163849 An. Burhanuddin Umar Lubis 06/05/2008 K 500.000.000,00 | Transfer Dari Bni Syariah An. Burhanudin Umar Lubis (Rekening 9153814507/05/2008 D 200.000.000,00 | Transfer Ke 146552434 An. Ditta Rachmawati 21/05/2008 D 100.000.000,00 | Tarik Tunai 21/05/2008 D 695.000.000,00 | Rtgs Ke Bca An. Abu Huraeroh Pada Tanggal 8 Juli 2009 Rekening Tutup, Dan Saldo Yang Tersisa Sebesar Rp1,25 Juta Digunakan/Ditransfer Untuk Membuka Rekening Baru Yaitu Nomor 150534508 An. Ybsjanuari 2014 Mutasi Rekening Cukup Aktif Dan Terdapat Beberapa Transaksi Dilakukan Dalam Nominal Relatif Signifikan Mencapai Ratusan Juta Rupiah. Adapun Transaksi Dimaksud Adalah Sebagai Berikut: Tanggal D/K Nominal Keterangan 28/04/2009 K 380.000.000,00 | Setor Tunai 05/06/2009 D 300.000.000,00 | Transfer Ke Burhanuddin Umar Lubis (1735134822/01/2010 K 180.000.000,00 | Setor Tunai 05/02/2010 D 150.000.000,00 | Rtgs Ke Pt Amalgam Indocorpora Di Bca 15/12/2010 K 210.000.000,00 | Setor Tunai 15/12/2010 D 210.000.000,00 | Ke 9154768 — Uang Jaminan Lelang Kpknl Jkt Iv 17/12/2010 K 179.860.000,00 | Setor Tunai 17/12/2010 D 179.8360.000,00 | Ke 9154768 /01/2012 K 215.000.000,00 | Setor Tunai 06/01/2012 D 215.000.000,00 | Transfer Ke 17351348 (Burhanuddin Umar Lubis30/01/2012 K 100.000.000,00 | Setor Tunai 30/01/2012 D 100.000.000,00 | Transfer Ke 17351348 (Burhanuddin Umar Lubis25/11/2013 K 100.000.000,00 | Setor Tunai 25/11/2013 D 100.000.000,00 | Transfer Ke 9154768 — Jaminan Lelang An. Ah Tanggal 31 Januari 2014 Rekening Tutup, Dengan Saldo Sebesar Rp0,00Adapun Profil Para Pihak Yang Bertransaksi Dengan Ah Adalah Sebagai Berikut: Nama Ditta Rachmawati Ttl Jakata, 21 Maret 1963 Nomor Ktp 0953046103630440 Jl. Kemuning Dalam I No. 37, Rt.O5 Rw.0O6, Pejaten Timuralamat Jakarta Selatan Pekerjaan Nama : Rekening Penampungan Lelang Kpknl Jakarta Iv Nama Pengurus : Salbiah Sh Nomor Npwp : 4562740023000 Alamat : Jl. Prapatan No. 10, Gambir, Senen, Jakarta Pusat No. Telepon : 021-3440910, 08129433983 Nama : Burhanuddin Umar Lubis H, Ma Ttl : Tap Selatan, 12 Maret 1935 Nomor Ktp I— 367405120336001 Jl. Ibnu Rusd Iil No. 1 Komp. Iain Rt O04 Rw 06 Ciputat-Tangerang Alamat Selatan Pekerjaan : Pns/Uin Rekening 0051033751 Atas Nama Ah Di Bca Kcu Wisma Milenia Dibuka Pada Tanggal 6 Juni 1989 Dan Telah Tutup Pada Tanggal 13 Desember 2002Rekening 2721223967 Atas Nama Ah Di Bca Kcp Gudang Peluru Dibuka Pada Tanggal 24 Juli 1998. Periode Januari 2004 Sampai Dengan 31 Desember 2013, Mutasi Rekening Cukup Aktif Dan Transaksi Dilakukan Dengan Nominal Bervariasi Mencapai Ratusan Juta Rupiah. Dana Masuk Pada Umumnya Terdiri Dari Setor Tunai, Transfer Dari Rekening Ybs Lainnya Maupun Dari Pihak Lain Antara Lain Siti Chair Rachman, Benny Jo Putra (Finance Control Pt Dexter Eurekatama — Freight Forwarding Hartono Dharma (Manager Pt Hrl Indonesia — Herbal)Sobari A, Novel Ali Dan Lain-Lain. Sedangkan Dana Keluar Pada Umumnya Terdiri Dari Tarik Tunai, Pembayaran Via Atm, Pembelian Via Edc, Transfer Ke Rekening Ybs Lainnya Dan Ke Pihak Lain Antara Lain Afifah, Hendrato, Siti Chair Rachman, Melanny, Lendrayani, Bpr Gitamakmur Utama, H Thalib, A. Syukri, Irawati Anwar, Hambali Sh, Andriana, Asnawi, Nina Chairunnisa, Dan Lain-Lain. Adapun Beberapa Transaksi Yang Dilakukan Dengan Nominal Signifikan Diatas Rp50 Juta Adalah Sebagai Berikut: Tanggal D/K Nominal Keterangan 13/01/2004 D 72.500.000,00 | Pb Ke Afifah 13/01/2004 D 50.000.000,00 | Pb Ke Hendrato 13/01/2004 K 72.500.000,00 | Setor Tunai 16/07/2004 K 72.500.000,00 | Setor Tunai Ybs 271/07/2004 D 50.000.000,00 | Pb Ke Siti Chair Rachman 05/10/2004 D 110.000.000,00 | Pb Ke Mellanny 05/10/2004 K 50.000.000,00 | Pb Dari Siti Chair Rachman 10/11/2004 K 150.000.000,00 | Setor Tunai Ybs 22/11/04 D 50.000.000,00 | Pb Ke Lendrayani 23/11/2004 D 70.000.000,00 | Pb Ke Bpr Gitamakmur Utama /20Be Doecumeamboraye Rewbesi Enith Kpire.Doc For Python03/12/2004 D 95.000.000,00 | Tarikan Tunai Dengan Buku 09/12/2004 K 53.300.000,00 | Setor Tunai 20/12/2004 D 51.000.000,00 | Tarik Tunai 22/12/2004 K 91.500.000,00 | Setor Tunai 24/06/2005 K 100.000.000,00 | Transfer Dari 2721016511 An. Siti Chair Rachman 28/06/2005 D 75.000.000,00 | Pb Ke Irawati Anwar 19/10/2006 K 150.000.000,00 | Pb Dari Rekening 2721339701 An. Ybs 09/11/2006 D 50.000.000,00 | Pb Ke H. Suudi Tabrani 13/11/2006 D 59.000.000,00 | Pb Ke Leni Tanudjaja 16/11/2006 K 94.000.000,00 | Setor Tunai Ybs 24/11/2006 K 75.000.000,00 | Pb Dari Rekening 2721339701 An. Ybs 01/12/2006 D 130.000.000,00 | Tarik Tunai 11/12/2006 D 65.000.000,00 | Pb Ke H Suudi Tabrani 07/05/2007 K 2.650.000.000,00 | Setor Tunai Ybs 02/07/2007 D 133.000.000,00 | Pb Ke Wina Nurulhayati 17/07/2007 D 135.000.000,00 | Tarikan Tunai 23/07/2007 D 75.000.000,00 | Pb Ke Bpr Artha Bersama 24/07/2007 D 51.500.000,00 | Pb Ke Bpr Artha Bersama 09/08/2007 D 1.100.000.000,00 | Tarik Tunai Ybs 20/08/2007 D 1.100.000.000,00 | Tarik Tunai Ybs 21/05/2008 K 150.000.000,00 | Pb Dari Rekening 2721339701 An. Ybs /Be Documamboraye Remtedemith2Spiasoloodor Python23/05/2008 D 95.000.000,00 | Tarik Tunai 14/08/2008 K 100.000.000,00 | Pb Dari 2721547258 An. Nina Chairunnisa 27/08/2008 D 97.500.000,00 | Tarik Tunai 06/02/2009 K 50.000.000,00 | Pb Dari Tourik Mahri 12/02/2009 K 50.000.000,00 | Pb Dari Tourik Mahri 16/02/2009 D 75.000.000,00 | Pb Ke Siti Chair Rachman 18/08/2009 D 50.000.000,00 | Pb Ke Nina Chairunnisa 21/08/2009 K 100.000.000,00 | Setor Tunai Ybs 21/08/2009 D 70.000.000,00 | Pb Ke Nina Chairunnisa 12/10/2009 D 65.631.000,00 | Pb Ke Net Sekuritas 28/10/2009 K 50.000.000,00 | Pemindahan Langsung Dari Tahapan 17/12/2009 K 74.000.000,00 | Pb Ke Siti Chair Rachman 17/12/2009 D 65.000.000,00 | Pb Ke Tourik Mahri 23/12/2009 K 75.000.000,00 | Setor Tunai 30/12/2009 D 75.000.000,00 | Pb Dari Rekening 2721339701 An. Ybs 29/10/2010 K 100.000.000,00 | Setor Tunai Ybs 29/10/2010 D 51.150.000,00 | Pb Ke Widianti 04/11/2010 D 50.000.000,00 | Pb Ke Widianti 04/11/2010 K 100.000.000,00 | Setor Tunai 09/11/2010 D 60.000.000,00 | Tarik Tunai 10/11/2010 K 85.000.000,00 | Setor Tunai Ybs /20Me Doecumembouayersatedayith Spire.Doc For Python13/12/2010 D 95.000.000,00 | Tarik Tunai 15/12/2010 D 95.000.000,00 | Tarik Tunai 06/01/2011 K 150.000.000,00 | Pb Dari Rekening 2721339701 An. Ybs 26/01/2011 K 50.000.000,00 | Setor Tunai 11/02/2011 K 200.000.000,00 | Setor Pemindahan 17/02/2011 D 100.000.000.00 | Tarik Tunai 18/02/2011 D 100.000.000,00 | Pb Ke Aisyawati Suseno 21/02/2011 K 200.000.000,00 | Setor Tunai 23/02/2011 K 200.000.000,00 | Pb Dari 2910433914 An. Aisyawati Suseno Kembalikan Uang Titipan 23/02/2011 D 200.000.000,00 | Pb Ke Aisyawati Suseno 01/03/2011 D 200.000.000,00 | Pb Ke Salmin Salim Aziz 23/03/2011 D 140.000.000,00 | Pb Ke Wahyudi Rudiyanto 01/04/2011 K 100.000.000,00 | Pb Dari Rekening 2721339701 An. Ybs 18/04/2011 K 70.000.000,00 | Setor Tunai 25/05/2011 K 128.698.500,00 | Pb Dari 7600750458 An. Handoyo Kurniadi S 23/06/2011 D 50.000.000,00 | Tarik Tunai 18/10/2011 D 250.000.000,00 | Pb Ke Tourik Mahri 18/10/2011 K 300.000.000,00 | Setoran Pemindahan 07/11/2011 D 95.000.000,00 | Pb Ke Nina Chairunnisa 07/11/2011 K 350.000.000,00 | Setoran Pemindahan 11/11/2011 D 75.000.000,00 | Pb Ke Arindo Wijanto — Pembayaran Rumah Document Was Created With Spire.Doc For Pythonpetojo 15/11/2011 K 300.000.000,00 | Pb Dari Rekening 2721339701 An. Ybs 15/11/2011 K 125.000.000,00 | Pb Dari Rekening 2721339701 An. Ybs 15/11/2011 D 265.000.000,00 | Pb Ke Arindo Wijanto 16/11/2011 K 140.000.000,00 | Setor Tunai 22/11/2011 D 60.000.000,00 | Pb Ke Arindo Wijanto 06/12/2011 D 110.000.000,00 | Pb Ke Arindo Wijanto 06/12/2011 D 500.000.000,00 | Pb Ke Hambali Sh 06/12/2011 K 250.000.000,00 | Setor Tunai Ybs 20/12/2011 D 55.000.000,00 | Pb Ke Nina Chairunissa 13/02/2012 K 1.227.542.000,00 | Setoran Pemindahan Dari Tambayong 17/02/2012 K 1.227.542.000,00 | Pb Dari 0033076999 An. Tambayong 12/03/2012 D 75.000.000,00 | Pb Ke Andi Ruslani — Pembayaran Rumah Batu Ja 30/04/2012 D 400.000.000,00 | Pb Ke Lufti Khoir 02/05/2012 D 293.000.000,00 | Pb Ke Lufti Khoir 14/05/2012 D 300.000.000,00 | Tarikan Pemindahan Ybs 11/06/2012 D 95.000.000,00 | Tarik Tunai 13/06/2012 D 146.800.000,00 | Pb Ke Hj Diah Fariati Bs 25/06/2012 D 50.000.000,00 | Pb Dari Rekening 2721339701 An. Ybs 04/07/2012 D 75.000.000,00 | Pb Ke Syah Rizal 13/08/2012 D 850.000.000,00 | Tarikan Pemindahan Ybs 16/10/2012 K 292.000.000,00 | Setor Tunai Flip Eduard Tombokan 20/02/2013 D 67.500.000,00 | Tarik Tunai /20Iee Doecumeamboaraya Rembeariwrein Spouro.Dee Lomdytinonpinjaman 28/02/2013 K 650.000.000,00 | Pb Dari 5680768481 An. Yulianti Fransiska 01/03/2013 K 300.000.000,00 | Rtgs Dari Neman Nasir Di Bni — Pembayaran Pinjaman 08/03/2013 K 150.000.000,00 | Rtgs Dari Ismeth Abdullah Di Bank Bukopin Pembayaran Hutang 13/03/2013 D 50.00.000,00 | Pb Ke Novel Rachman 22/03/2013 K 50.000.000,00 | Pb Dari Novel Rachman 26/03/2013 D 50.000.000,00 | Pb Ke Novel Rahcman 02/04/2013 K 59.500.000,00 | Pb Dari 7150624007 An. Asnawi 05/04/2013 D 50.000.000,00 | Pb Ke Novel Rachman 05/04/2013 D 50.000.000,00 | Tarik Tunai 09/04/2013 K 910.000.000,00 | Pb Dari 0350681540 An. Arindo Wijanto"

jumlah_kata = len(text.split())
print(f"\nJumlah kata dalam teks asli: {jumlah_kata}")

# Chunk the text
max_tokens = 500  # Adjust as needed to be below the model's token limit
text_chunks = chunk_text_to_tokens(text, max_tokens)

# Summarize each chunk and combine
summaries = [summarizer(chunk)[0]['summary_text'] for chunk in text_chunks]
combined_summary = ' '.join(summaries)

# NER text
ner_results = ner_pipeline(text)
ner_results

#Merge NER
merged_results = merge_ner_results(ner_results)
merged_results
print ("============================================")
print("Summary:\n", combined_summary)
jumlah_kata_summary = len(combined_summary.split())
print(f"\nJumlah kata dalam teks summary: {jumlah_kata_summary}")
print("============================================")
# Extracting 5W1H information
who = []
when = []
where = []
what = []
how_much = []
for entity in merged_results:
    if (entity['entity_group'] == 'PER') | (entity['entity_group'] == 'ORG') | (entity['entity_group'] == 'NOR'):
        who.append(entity['word'])
    elif (entity['entity_group'] == 'LOC') | (entity['entity_group'] == 'GPE') | (entity['entity_group'] == 'FAC') | (entity['entity_group'] == 'REG'):
        where.append(entity['word'])
    elif (entity['entity_group'] == 'TIM') | (entity['entity_group'] == 'DAT'):
        when.append(entity['word'])
    elif (entity['entity_group'] == 'EVT') | (entity['entity_group'] == 'PRC'):
        what.append(entity['word'])
    elif (entity['entity_group'] == 'MON') | (entity['entity_group'] == 'QTY'):
        how_much.append(entity['word'])
print("Who:", ", ".join(who))
print("When", ", ".join(when))
print("Where:", ", ".join(where))
print("What:", ", ".join(what))
print("How Much:", ", ".join(how_much))

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 09:29:46

Jumlah kata dalam teks asli: 2108
Summary:
 transaksi keuangan mencurigakan an. abu huraeroh 1. kasus posisi terinformasi abu jibaeroh / ah ( 49 tahun ) adalah pegawai negeri sipil pada pemerintah provinsi dki dengan jabatan kas nomor 1. 0107 1964. 0463 ( e33 )3 ( d. 0963 ) 2008 ( btn ), bminomor identitas pns - kabag keuangan bogasari flour flour mills ( industri makananzi zlujtujatuzil disuma inina yielman unntila dengan nama ah di bank panin kcp teb kependudukan rekening cukup aktif dan terdapat beberapa transaksi dilakukan dalam nominal relatif signifikan mencapai ratusan juta rupiah. namun transaksi tersebut adalah sebagai berikut. tahun ini, mutasi rekening cukup aktif dan transaksi dilakukan dengan nominal bervariasi mencapai ratusan juta rupiah. dana masuk pada umumnya terdiri dari setor tunai, pembayaran via atm, pembelian via edc, pembelian sebanyak 3. 000. 000, 00 tarikan tunai dengan buku 09 / 12 / 2004 k. 300. 00 tarik tunai 20 / berikutnya 20 / 20

In [14]:
# test ketiga
text="/12/14/Sr-M Hasil Analisis Transaksi Keuangan Atas Nama Jen Prawiro Tegoeh 1. Ringkasan Kasus Terdapat Surat Permintaan Informasi Direktur Jenderal Pajak Nomor : Sr570/Pj1.05/2014 Tanggal 7 Juli 2014, Perihal Permintaan Informasi Terkait Dugaan Tindak Pidana Perpajakan Dengan Terlapor Jen Prawiro Tegoeh Yang Saat Ini Sedang Dalam Proses Pengembangan Dan Analisis Di Direktorat Intelijen Dan Penyidikan, Direktorat Jenderal Pajak2. Profil Pihak Nama : Jen Prawiro Tegoeh Ktp : 3573021908500001 Tempat Lahir : Malang Umur/Tanggal Lahir : 64 Tahun / 19 Agustus 1950 Jenis Kelamin : Laki-Laki Kebangsaan : Indonesia Tempat Tinggal : Jl Bukit Berbunga No.25 Rt.003 Rw.003, Kota Batu, Jawa Timur Agama : Islam Informasi Rekening : Termuat Dalam Hasil Analisis 3. Rincian Informasi Berdasarkan Penelusuran Di Database Ppatk, Dapat Diketahui Informasi Transaksi Rekening Milik Jen Prawiro Tegoeh Di Pt. Bank Internasional Indonesia Tbk (Bii Pt. Bank Panin Tbk, Pt. Bank Central Asia Tbk, Centratama Nasional Bank (Cnb Pt. Bank Cimb Niaga Tbk, Pt. Bank Mandiri (Persero) Tbk, Dan Pt. Bank Btpn Tbk Sebagai Berikut : A. Rekening Atas Nama Jen Prawiro Tegoeh Di Bank Internasional Indonesia 1. Informasi Rekening No. 1-052-800470 Atas Nama Jen Prawiro Tegoeh Di Bii Selama Periode Januari 2010 S.D. Desember 2012: Berdasarkan Database Ppatk, Tidak Ditemukan Transaksi Dengan Nilai Signifikan Selama Periode Januari 2010 - Desember 2012, Transaksi Yang Terjadi Hanya Pengkreditan Bunga Tabungan Dan Pendebetan Pajak2. Informasi Rekening No. 1052800795 Atas Nama Donny Wahyudi Tegoeh Or Jen Prawiro Tegoeh (Hubungan Ayah Dan Anak) Di Bii Selama Periode Januari 2010 S.D. Desember 2012: Tidak Ditemukan Transaksi Dengan Nilai Signifikan Selama Periode Januari 2010 Desember 2012, Transaksi Yang Terjadi Hanya Pemindahbukuan Melalui Atmsetoran Tunai Dengan Nominal Rp.7.500.000 Pengkreditan Bunga Tabungan Dan Pendebetan Pajakb. Rekening Atas Nama Jen Prawiro Tegoeh Di Bank Panin C. Rekening Atas Nama Jen Prawiro Tegoeh Di Bank Central Asia (Bca A. Berikut Disampaikan Rekapitulasi Transaksi Berdasarkan Jenis Transaksi: D/K Jenis Transaksi ?:::::::: Jumlah (Rpbiaya Adm 51 520,000.00 Biaya Atm Lain (Switching) 1 5,000.00 Db Otomatis 3 825,000.00 Pajak Bunga 18 181,935.56 Pembayaran Bcacard Via Atm 3 1,800,000.00 Pembayaran Via Atm 31 40,459,822.00 D Pembelian Via Edc 51 54,835,158.00 Tarikan Tunai Via Atm 14 24,000,000.00 Iransfer Cust Ke Non-Custmia 1Transfer Via Atm Ke Giro 9 36,904,000.00 Transfer Via Atm Ke Tahapan 47 157,499,300.00 Transfer Via Atm Ke Tapres 2 5,400,000.00 D Total 231 322,930,215.56 K Bunga 36 991,305.54 _Iid_Ehkx;Zqhan Langsung Dari 4 130,389,634.00 Ziie]R'Sran Tabungan Statement & 3 30,249,000.00 Frekuensi D/K Jenis Transaksi Transaksi Jumlah (Rptransfer Via Atm Dari Giro 1 340,000.00 Transfer Via Atm Dari Tahapan 6 26,040,100.00 Transfer Via Ib Dari Tahapan 2 124,003,825.00 Transfer Via Mb Dari Tahapan 1 11,978,000.00 K Total 53 323,991,864.54 Grand Total 284 646,922,080.10 B. Berikut Adalah Beberapa Pihak Yang Menerima Transfer Dari Jen Prawiro Tegoeh: Pihak Penerima :_:_::::::: Jumlah (Rpace Hardware Indonesia Tbk Pt 1 186,200.00 Achmad 1 8,250,000.00 Albert Budijono 1 3,482,000.00 Amelia Kristanti 2 1,958,000.00 Anggreani Natadihardja 1 97,550.00 Annastasia Tienneke Kusumawati 2 1,485,000.00 Astra International Tbk Pt 1,322,237.00 Asuransi Bintang Tbk Pt 5,582,000.00 Bank Cimb Niaga Tbk Pt 2,800,000.00 Bank Mega Tbk Pt 4,900,000.00 Citibank Na 7,650,000.00 Dewi Komalasari 17,547,000.00 Dewi Soetedjo 16,899,500.00 Indomarco Prismatama Pt 2 1 2 6 4 2 4 Donny Wahyudi Tegoeh 3 5,740,000.00 Eddy Prakoso Tegoeh 1 11,978,000.00 Fifi Veronita 2 980,000.00 Gerry Newan Hargono 1 5,000,000.00 Giyastuti S 1 2,000,000.00 Gramedia Asri Media Pt 1 284,200.00 Gunawan Tanojo 1 2,365,000.00 Harianto Tedja 1 900,000.00 Hartini 1 350,000.00 Hartono Suprapto 2 3,412,000.00 Hendro Teguh 3 195,630,959.00 Hero Supermarket Pt 4 1,898,633.00 Heru Retmono 2 422,800.00 1 188,000.00 Frekuensi Pihak Penerima Transaksi Jumlah (Rpindra Setiyadi 1 5,000,000.00 Ingrid Angelica 1 180,000.00 Ira Susanti 1 340,000.00 Iwan Singonegoro 1 1,074,000.00 Jen Prawiro Tegoeh 1 20,000,000.00 Joehannes Harianto 2 956,523.00 Jonathan 1 890,000.00 Jos Tjekosugiarto 2 729,960.00 Lana Kusuma 1 306,525.00 Liliany Soedarko Drg 3 1,351,400.00 M Yasin Ferdiansyah Se 7 2,057,960.00 Megawati Soetejo 1 979,000.00 Meliana Sukowijoyo 1 10,000,000.00 Meriyati 4 2,085,170.00 Milawati Soehardjo 1 10,000,000.00 Nacrowi 3 47,448,700.00 Ng Mie Lie 1 20,000,000.00 Re.Doesdfwebythonnovita 13,050,000.00 Oetoyo Abid 1,135,000.00 Ong Djoeng 2,397,500.00 Penampungan Pbb 56,908.00 Petrus Petra Prakasa Setiawan 1,417,500.00 Putu Jindriasih Prasetya 9,000,000.00 Ratnasari Hartanu 33,218,000.00 Rudy Sudrajat 1,343,300.00 Sarinah Persero Pt 212,000.00 Sewu Gunawan 525,000.00 Slamat Rahardjo Kurnia 7,046,500.00 Soesiana Moelia 317,500.00 Sonny Subagio Haliman 17,104,900.00 Sudarto Oeiria Dr 27,069,000.00 Tan Merlin Tanumihardjo 15,936,000.00 The Hongkong Shanghai Banking 23,800,000.00 Vera 3,125,000.00 Vonny Se 137,500.00 Welly Tanubrata 125,000.00 Wisata Lintas Benua Pt 1,785,000.00 Xl Axiata Tbk Pt 1,031,274.00 Yay Senilukis Indonesia 100,000.00 Hiojnh/Hiju I N/Hio Njin/Nhr/Rjrjo |Hih In Hihi U Pihak Penerima :_:_::::::: Jumlah (Rpyay Sumber Wangi Mitra Bhakti 1 906,000.00 Tidak Diketahui 132 37,294,881.10 Grand Total 284 646,922,080.10 Berikut Diuraikan Rincian Transaksi Dengan Nilai 2 10,000,000.00: Tanggal Uraian Transaksi D/K Jumlah (Rp07/04/2010 Transfer Via Ib Dari Tahapan K 87,716,307.00 11/10/2010 Pemindahan Langsung Dari Tahapan K 71,627,134.00 12/07/2010 Transfer Via Ib Dari Tahapan K 36,287,518.00 25/10/2011 Pemindahan Langsung Dari Tahapan K 25,000,000.00 19/10/2011 Pemindahan Langsung Dari Tahapan K 20,000,000.00 11/05/2012 Transfer Via Atm Ke Tahapan D 20,000,000.00 24/05/2012 Setoran Tabungan Statement & Giro K 20,000,000.00 09/11/2011 Transfer Via Atm Ke Giro D 17,047,000.00 12/08/2010 Transfer Via Atm Ke Tahapan D 16,380,000.00 07/11/2011 Transfer Via Atm Ke Tahapan D 15,936,000.00 25/11/2011 Transfer Via Atm Dari Tahapan K 15,936,000.00 Galil Do Bg Na Ban Gne Bany Hehasani 1 E.Dor 1Or Py3D3 Gbb.Co 01/12/2011 Transfer Via Mb Dari Tahapan K 11,978,000.00 03/11/2010 Transfer Via Atm Ke Tahapan D 10,000,000.00 09/11/2010 Transfer Via Atm Ke Tahapan D 10,000,000.00 2. Informasi Rekening No. 3857588889 Atas Nama Jen Prawiro Tegoeh Di Bca Selama Periode Januari 2010 S.D. Desember 2012: A.Berikut Disampaikan Rekapitulasi Transaksi Berdasarkan Jenis Transaksi: D/K Jenis Transaksi Frekuensi Transaksi Jumlah (Rpd Biaya Adm 3 24,552.20 D Total 3 24,552.20 Grand Total 3 24,552.20 B.Berikut Adalah Beberapa Pihak Yang Menerima Transfer Dari Jen Prawiro Tegoeh : Pihak Penerima Frekuensi Transaksi Jumlah (Rptidak Diketahui 3 24,552.20 Grand Total 24,552.20 3. Informasi Rekening No. 8160758857 Atas Nama Jen Prawiro Tegoeh Di Bca Selama Periode Januari 2010 S.D. Desember 2012: A. Berikut Disampaikan Rekapitulasi Transaksi Berdasarkan Jenis Transaksi: D/K Jenis Transaksi :_:::::::: Jumlah (Rpbiaya Adm 36 370,000.00 Biaya Auto Cr Merchant Card Center 19 203,650.00 Denda Thd Bca Prioritas 14 3,500,000.00 Pajak Bunga 36 90,359.47 D Pembelian Via Edc 3 2,288,140.00 Zi;I:Icindahan Langsung Tabungan Ke 1 3,901,464.00 Transfer Via Atm Ke Giro 3 18,423,608.00 Transfer Via Atm Ke Tahapan 2 752,500.00 D Total 114 29,529,721.47 Bunga 36 451,797.50 Kr Otomatis Debit Card Settlement 63,056,758.00 £Yfhg)Nnk-Lainnya 14 3,500,000.00 Transfer Via Atm Dari Tahapan 2 1,200,000.00 K Total 131 83,957,388.50 Grand Total 245 113,487,109.97 B. Berikut Adalah Beberapa Pihak Yang Menerima Transfer Dari Jen Prawiro Tegoeh : Pihak Penerima Frekuens_I Jumlah (Rptransaksi C Ery Retnowati 1 200,000.00 Donny Wahyudi Tegoeh 2 15,966,000.00 Helly Leo Garlan 1 1,000,000.00 Jen Prawiro Tegoeh 1 3,901,464.00 Jos Tjekosugiarto 1 1,258,040.00 Mariono Junaidi 1 180,000.00 Prodia Widyahusada Pt 1 723,600.00 Rahayu Arumdhani I Pt 1 2,457,608.00 Slamat Rahardjo Kurnia 1 572,500.00 Tidak Diketahui 235 87,227,897.97 Grand Total 245 113,487,109.97 C. Berikut Diuraikan Rincian Transaksi Dengan Nilai 25,000,000.00: Tanggal Uraian Transaksi D/K Jumlah (Rp27/11/2012 | Transfer Via Atm Ke Giro D 8,950,000.00 19/12/2012 | Transfer Via Atm Ke Giro D 7,016,000.00 4. Informasi Rekening No. 8160848333 Atas Nama Jen Prawiro Tegoeh Di Bca Selama Periode Januari 2010 S.D. Desember 2012: A. Berikut Disampaikan Rekapitulasi Transaksi Berdasarkan Jenis Transaksi: Frekuensi D/K Jenis Transaksi Transaksi Jumlah (Rpbiaya Adm 36 327,760.00 Pajak Bunga 36 1,172,009.10 D Tarikan Bank Note 1 27,435,000.00 Tarikan Pemindahan Statement 8 11,234,092,625.00 Transfer Via Atm Ke Bcadollar 12 57,755,200.00 D Total 93 11,320,782,594.10 Bunga 36 5,860,873.20 . . Tematis Remittances D & Giro 3 5,799,093,973.80 Transfer Bunga Deposito 23 3,469,510.35 K Total 65 11,924,713,085.55 Grand Total 158 23,245,495,679.65 B. Berikut Adalah Beberapa Pihak Yang Menerima Transfer Dari Jen Prawiro Tegoeh : Pihak Penerima :.:::::::: Jumlah (Rphajanto Segaradjaja Drs 12 57,755,200.00 Jen Prawiro Tegoeh 7 688,695,444.05 Ratnasari Hartanu 19 5,113,868,040.10 Tidak Diketahui 120 17,385,176,995.50 Grand Total 158 23,245,495,679.65 C. Berikut Diuraikan Rincian Transaksi Dengan Nilai 2 10,000,000.00: Tanggal Uraian Transaksi D/K J — Jumlah (Rp10/05/2010 | Tarikan Pemindahan Statement D | 5057,250,000.00 19/04/2010 | Se Oran Tabungan Statement & K | 4985,295,000.00 12/12/2012 | Kr Otomatis Remittances (Online) | K | 3.702,539,792.00 22/01/2010 Tarikan Pemindahan Statement Tanggal Uraian Transaksi D/K Jumlah (Rp13/06/2011 | Tarikan Pemindahan Statement 2,557,542,625.00 28/03/2011 | Kr Otomatis Remittances (Online) 1,307,075,700.00 01/06/2011 | Kr Otomatis Remittances (Online) 1,106,673,236.20 935,000,000.00 22/01/2010 Tarikan Pemindahan Statement 935,000,000.00 D K K D D 22/01/2010 | Tarikan Pemindahan Statement D 701,250,000.00 25/03/2010 Ziieiisran Tabungan Statement & K 685,913,317.55 25/03/2010 | Tarikan Pemindahan Statement D 548,700,000.00 22/01/2010 | Tarikan Pemindahan Statement D 467,500,000.00 13/06/2011 2L_Gran Tabungan Statement & K 127,885,656.25 02/03/2012 | Tarikan Pemindahan Statement D 31,850,000.00 19/05/2010 | Tarikan Bank Note D 27,435,000.00 17/05/2010 | Transfer Via Atm Ke Bcadollar D 18,250,000.00 17/05/2010 | Transfer Via Atm Ke Bcadollar D 11,406,250.00 5. Informasi Rekening No. 8160784785 Atas Nama Jen Prawiro Tegoeh Frekuensi D/K Jenis Transaksi Transaksi Jumlah (Rpauto Debet Icossme 3 11,250,000.00 Biaya Adm 36 1,080,000.00 Db Otomatis Biaya Kliring 2 4,000.00 Nd-Buku Cek/Bg 5 500,000.00 Nd-Lainnya 11 22,387,212.50 D Pajak Bunga 11 219,859.71 Pelunasan Bunga Pinjaman 4 2,033,541.43 Pembayaran Pinjaman 31 275,983,433.79 Tarikan Giro Lokal 124 25,551,179,150.00 Tarikan Pemindahan Dgn Warkat 1 27,000,000.00 Tolakan Kliring Same Day 1 62,500,000.00 D Total 229 25,954,137,197.43 Auto Credit Icossme 1 1,000,000,000.00 Bunga 12 1,102,684.10 Kr Otomatis Rtgs 15 14,752,350,000.00 K _Iid_Ezi'Iaiezqhan Langsung Dari 7 1,093,200,000.00 [S)I'L;Oran Kliring Luar Kota Next 4 1,000,000,000.00 Setoran Kliring Titipan Same Day 1 62,500,000.00 Frekuensi D/K Jenis Transaksi Transaksi Jumlah (Rp28Ran Tabungan Statement & 15 4,929,261,441.00 Setoran Tunai Giro 11 177,000,000.00 Transfer Via Atm Dari Giro 1 12,153,400.00 Transfer Via Atm Dari Tahapan 21 972,258,000.00 Transfer Via Mb Dari Tahapan 1 62,500,000.00 K Total 86 24,062,325,525.10 Grand Total 315 50,016,462,722.53 B. Berikut Adalah Beberapa Pihak Yang Menerima Transfer Dari Jen Prawiro Ythontegoeh : Pihak Penerima :;::::::: Jumlah (Rpabdul Havid 1 228,500,000.00 Aland Soesanto 1 100,000,000.00 Alexander Tambayong 1 62,500,000.00 Deny Wijaya 5 365,000,000.00 . . | Donny Wahyudi Tegoeh L. . 12.153,400.00 Fenny Hendrijawati 2 41,500,000.00 Gunawan Tanojo 1 10,000,000.00 Hartono 7 477,250,000.00 Jen Prawiro Tegoeh 6 4,712,871,441.00 Kastadji 3 354,700,000.00 Muhamad Novel 1 143,000,000.00 Ratnasari Hartanu 11 232,498,000.00 Tuty Irawati 1 210,000,000.00 Winarto 1 12,900,000.00 (Blank) 270 42,814,089,881.53 Grand Total 315 50,016,462,722.53 C. Berikut Diuraikan Rincian Transaksi Dengan Nilai 2500,000,000.00: Tanggal Uraian Transaksi D/K Jumlah (Rp12/04/2010 | Kr Otomatis Rtgs K | 6,924,850,000.00 13/04/2010 | Tarikan Giro Lokal D | 6924,850,000.00 06/04/2010 | Tarikan Giro Lokal D | 3664,000,000.00 29/07/2011 | Kr Otomatis Rtgs K | 2,000,000,000.00 12/01/2011 | Tarikan Giro Lokal D 1,300,000,000.00 16/07/2012 Z'Iefisran Tabungan Statement & K 1,124,700,000.00 31/03/2010 | Kr Otomatis Rtgs K 1,000,000,000.00 Tanggal Uraian Transaksi D/K Jumlah (Rp16/08/2010 [S)Iloran Kliring Luar Kota Next K 1,000,000,000.00 31/12/2010 | Tarikan Giro Lokal D 1,000,000,000.00 12/10/2012 | Auto Credit Icossme K 1,000,000,000.00 25/03/2010 Ziie;Gran Tabungan Statement & K 913,951,087.00 25/03/2010 Ziieqsran Tabungan Statement & K 913,951,087.00 01/04/2010 | Kr Otomatis Rtgs K 800,000,000.00 19/07/2012 Ziie;Gran Tabungan Statement & K 753,500,000.00 06/09/2010 | Tarikan Giro Lokal D 7102,027,500.00 08/07/2011 | Kr Otomatis Rtgs K 700,000,000.00 25/03/2010 Ziieqsran Tabungan Statement & K 548,340,000.00 21/07/2011 | Tarikan Giro Lokal D 520,000,000.00 21/12/2011 | Kr Otomatis Rtgs K 500,000,000.00 28/12/2011 | Tarikan Giro Lokal D 500,000,000.00 29/12/2011 | Kr Otomatis Rtgs K 500,000,000.00 A. Berikut Disampaikan Rekapitulasi Transaksi Berdasarkan Jenis Transaksi: D/K | Jenis Transaksi _::::::: Jumlah (Rpd Biaya Adm 7 108,363.92 Pajak Bunga 7 50,683.07 Tarikan Bank Note 1 14,977,200.00 Tarikan Pemindahan Statement 2 1,873,680,000.00 D Total 17 1,888,816,246.99 K Bunga 7 253,338.17 Fsniix?Eiv)Iatis Remittances 1 2,114,827,000.00 Setoran Bank Note 1 1,475,400.00 K Total 9 2,116,555,738.17 Grand Total 26 4,005,371,985.16 B. Berikut Adalah Beberapa Pihak Yang Menerima Transfer Dari Jen Prawiro Tegoeh : . . Frekuensi Pihak Penerima Transaksi Jumlah (Rptidak Diketahui 26 4,005,371,985.16 Grand Total 26 4,005,371,985.16"

from datetime import datetime

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)
jumlah_kata = len(text.split())
print(f"\nJumlah kata dalam teks asli: {jumlah_kata}")

# Chunk the text
max_tokens = 500  # Adjust as needed to be below the model's token limit
text_chunks = chunk_text_to_tokens(text, max_tokens)

# Summarize each chunk and combine
summaries = [summarizer(chunk)[0]['summary_text'] for chunk in text_chunks]
combined_summary = ' '.join(summaries)

# NER text
ner_results = ner_pipeline(text)
ner_results

#Merge NER
merged_results = merge_ner_results(ner_results)
merged_results
print ("============================================")
print("Summary:\n", combined_summary)
jumlah_kata_summary = len(combined_summary.split())
print(f"\nJumlah kata dalam teks summary: {jumlah_kata_summary}")
print("============================================")
# Extracting 5W1H information
who = []
when = []
where = []
what = []
how_much = []
for entity in merged_results:
    if (entity['entity_group'] == 'PER') | (entity['entity_group'] == 'ORG') | (entity['entity_group'] == 'NOR'):
        who.append(entity['word'])
    elif (entity['entity_group'] == 'LOC') | (entity['entity_group'] == 'GPE') | (entity['entity_group'] == 'FAC') | (entity['entity_group'] == 'REG'):
        where.append(entity['word'])
    elif (entity['entity_group'] == 'TIM') | (entity['entity_group'] == 'DAT'):
        when.append(entity['word'])
    elif (entity['entity_group'] == 'EVT') | (entity['entity_group'] == 'PRC'):
        what.append(entity['word'])
    elif (entity['entity_group'] == 'MON') | (entity['entity_group'] == 'QTY'):
        how_much.append(entity['word'])
print("Who:", ", ".join(who))
print("When", ", ".join(when))
print("Where:", ", ".join(where))
print("What:", ", ".join(what))
print("How Much:", ", ".join(how_much))

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 09:42:44

Jumlah kata dalam teks asli: 1945
Summary:
 / 12 / 14 / sr - m hasil analisis transaksi keuangan atas nama jen prawiro tegoeh 1. prediksi kasus terdapat surat permintaan informasi terkait dugaan tindak pidana perpajakan dengan terlapor jen praw rabu ( 5 / 10 ), 000. 00 pajak bunga bunga 18 181, 935. 56 pembayaran tunai via atm 3, 1, 800, 2, 2. 000. prakoso tegoeh 1 11, retinai 2 980, 000. 00 gerry newan hargono 1, 3. 000. sementara itu, 600 ribu rupiah. , 900. 00 pu merlin tanumihardjo 15, 2, 936, 000. senin ( 10 / 10 ) transfer via atm ke tahapan d 271, 091. informasi rekening no. 3868 orthiro tegoeh di bca selama periode januari 2010 s. desember 2012 : a. berikut disampaikan rekapitulasi transaksi berdasarkan jenis transaksi. 1 723, 600. 00 rahayu arumdhani i pt 1 2, 457, 608. 00 tidak diketahui 235 87, 227, 487. 97 c. 97 04 / 2010 se oran tabungan statement & 135, 304, 000. 00 12 / 2010 tarikan pemindahan statement tanggal 730 transaksi d. 702, 539. 00 22 se